# Federated AutoEncoder  with Flex  for Anomaly Detection

First we do all the imports needed.

In [ ]:
from flexanomalies.models import AutoEncoder
from flexanomalies.utils.load_data import load_and_split_dot_mat, federate_data
from flexanomalies.pool.decorators_autoencoder import build_server_model_ae, copy_model_to_clients_ae,train_ae, aggregate_ae, set_aggregated_weights_ae, weights_collector_ae
from flexanomalies.utils.save_results import save_experiments_results
from flex.pool import FlexPool
from flexanomalies.utils.metrics import *


In [2]:
model_params = {
    "epochs": 5,
    "input_dim": 9,
    "batch_size": 32,
    "neurons": [8, 4, 8],
    "hidden_act": ["relu", "relu", "relu"],
    "contamination": 0.1,
}

In [3]:
X_train, X_test, y_train, y_test = load_and_split_dot_mat("flexanomalies/datasets/data/shuttle.mat", 0.3)

In [4]:
model = AutoEncoder(**model_params)

2023-12-08 20:36:18.965282: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-12-08 20:36:18.965299: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-12-08 20:36:18.965312: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (beatriz-IdeaPad-3-15ITL6): /proc/driver/nvidia/version does not exist
2023-12-08 20:36:18.965491: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
flex_dataset = federate_data(5, X_train, y_train)

In [7]:
pool = FlexPool.client_server_pool(
        fed_dataset=flex_dataset,
        server_id="autoencoder_server",
        init_func=build_server_model_ae,
        model=model,
    )

In [8]:
for i in range(3):
    print(f"\nRunning round: {i}\n")
    pool.servers.map(copy_model_to_clients_ae, pool.clients)
    pool.clients.map(train_ae)
    pool.aggregators.map(weights_collector_ae, pool.clients)
    pool.aggregators.map(aggregate_ae)
    pool.aggregators.map(set_aggregated_weights_ae, pool.servers)
output_model = pool.servers._models["autoencoder_server"]["model"]
output_model.evaluate(X_test, y_test)



Running round: 0

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
......dense_3
.........vars
............0
............1
...optimizer
......vars
.........0
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-12-08 20:36:25         2217
variables.h5                                   2023-12-08 20:36:25        17680
metadata.json                                  2023-12-08 20:36:25           64
Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-12-08 20:36:24         2217
variables.h5                                   2023-12-08 20:36:24        17680
metadata.json          

In [9]:
output_model.result_metrics_

{'Accuracy': '96.959%',
 'Precision': '0.706',
 'F1': '0.823',
 'Recall': '0.986',
 'AUC_ROC': '0.977'}

In [10]:
save_experiments_results(
    'autoencoder',
    output_model,
    'test_autoencoder_notebook',
    model_params,
    'shuttle.mat',
    5,
    3,
    0.3,
)